<a href="https://colab.research.google.com/github/Sjoerd-de-Witte/Machine-Learning-2023/blob/main/4_1_Cleaning_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
!gdown -O /tmp/ml.py 174lBNvDBJSVWs3OpNL3a68cnhWIcWYuY
%run /tmp/ml.py

Downloading...
From: https://drive.google.com/uc?id=174lBNvDBJSVWs3OpNL3a68cnhWIcWYuY
To: /tmp/ml.py
100% 1.31k/1.31k [00:00<00:00, 4.54MB/s]


# Data Cleaning

In this notebook, we will demonstrate the most common cleaning techniques, and have exercises for you to complete. We will use a tiny dataset.

In [91]:
from pipetorch.data import realestate
import pandas as pd
import numpy as np

# Load and inspect the data

What issues do you see?

Ignore for this exercise:
- There is no clear goal or target variable defined
- It is unclear whether this dataset is representative

You will still have to check:
- is the data information rich?
- is the data reliable?
- is the data clean (no missing values, numeric, issues solved)

In [92]:
df = realestate()

In [93]:
df

,PID,ST_NUM,ST_NAME,OWN_OCCUPIED,NUM_BEDROOMS,NUM_BATH,SQ_FT
0,100001000.0,104.0,PUTNAM,Y,3,1,1000
1,100002000.0,197.0,LEXINGTON,N,3,1.5,--
2,100003000.0,NaN,LEXINGTON,N,NaN,1,850
3,100004000.0,201.0,BERKELEY,12,1,NaN,700
4,NaN,203.0,BERKELEY,Y,3,2,1600
5,100006000.0,207.0,BERKELEY,Y,NaN,1,800
6,100007000.0,NaN,WASHINGTON,NaN,2,HURLEY,950
7,100008000.0,213.0,TREMONT,Y,1,1,NaN
8,100009000.0,215.0,TREMONT,Y,na,2,1800


# Clean this dataset


In [97]:
# Drop meaningless columns
# Properly convert garbage to missing values
# resolve missing values
df = realestate(na_values=['n/a', 'na', '--'])
df.OWN_OCCUPIED.replace('12', np.nan, inplace=True)
df['NUM_BATH'] = pd.to_numeric(df.NUM_BATH.replace('HURLEY', np.nan))
df['ST_NAME'] = df.ST_NAME.replace('PUTNAM', np.nan)

from sklearn.impute import SimpleImputer
imp = SimpleImputer()
np.set_printoptions(suppress=True)
df[['NUM_BEDROOMS', 'NUM_BATH', 'SQ_FT']] = imp.fit_transform(df[['NUM_BEDROOMS', 'NUM_BATH', 'SQ_FT']])
df['OWN_OCCUPIED'].fillna(df['OWN_OCCUPIED'].mode()[0], inplace=True)
df['ST_NAME'].fillna(df['ST_NAME'].mode()[0], inplace=True)

df = df.drop(columns=['PID', 'ST_NUM'])

In [98]:
df

,ST_NAME,OWN_OCCUPIED,NUM_BEDROOMS,NUM_BATH,SQ_FT
0,BERKELEY,Y,3.000000,1.000000,1000.0
1,LEXINGTON,N,3.000000,1.500000,1100.0
2,LEXINGTON,N,2.166667,1.000000,850.0
3,BERKELEY,Y,1.000000,1.357143,700.0
4,BERKELEY,Y,3.000000,2.000000,1600.0
5,BERKELEY,Y,2.166667,1.000000,800.0
6,WASHINGTON,Y,2.000000,1.357143,950.0
7,TREMONT,Y,1.000000,1.000000,1100.0
8,TREMONT,Y,2.166667,2.000000,1800.0


# Categorical values

All values must be numeric. To ST_NAME and OWN_OCCUPIED to numeric values, we should create dummy variables. The way this works is that for every categorical value (but one) a new column is created that contains a 0 or 1 value to indicate whether that property is labelled with that category. We can remove one category, since when all other categories are labelled as 0, it must be the category that we left out, therefor we still have the same information.

There are many bad suggestions about handling categorical values online, the proper way to use dummy variables is to:
- only use the categorical values that are in the training set, to avoid leaking info from the validation/test set
- to assign a separatxe category to missing values
- to consider values in the valid/test set that are not in the train set as missing values

Unfortunately, get_dummies is only suitable for handling a single data set and useless for dealing with a validation. You should transform you categorical values in the correct way using SKLearn's OneHotEncoder.

In [100]:
pd.get_dummies(df[['OWN_OCCUPIED', 'ST_NAME']], columns=['ST_NAME'], drop_first=True)

,OWN_OCCUPIED,ST_NAME_LEXINGTON,ST_NAME_TREMONT,ST_NAME_WASHINGTON
0,Y,0,0,0
1,N,1,0,0
2,N,1,0,0
3,Y,0,0,0
4,Y,0,0,0
5,Y,0,0,0
6,Y,0,0,1
7,Y,0,1,0
8,Y,0,1,0


In [ ]:
# Replace ST_NAME and OWN_OCCUPIED with dummy variables using SKLearn's OneHotEncoder
pd.get_dummies(df[['ST_NUM', 'ST_NAME']], columns=['ST_NAME'], drop_first=True)

array([[3.        , 1.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 1.        ],
       [3.        , 1.5       , 0.        , 1.        , 0.        ,
        0.        , 0.        , 1.        , 0.        ],
       [0.        , 1.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 1.        , 0.        ],
       [1.        , 1.35714286, 1.        , 0.        , 0.        ,
        0.        , 0.        , 1.        , 0.        ],
       [3.        , 2.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 1.        ],
       [0.        , 1.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 1.        ],
       [2.        , 1.35714286, 0.        , 0.        , 0.        ,
        0.        , 1.        , 1.        , 0.        ],
       [1.        , 1.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 1.        ],


In [ ]:
halt_notebook()